#### 단어 임베딩 : 각 단어를 벡터로 표현. 
####             : - 동음이의어 구분X -> 동일한 벡터값으로 표현.
####             : + 문장 임베딩에 비해 학습 방법 간단.

In [26]:
# 1. 원-핫 인코딩(one-hot encoding)
# -> 원-핫 벡터 = 희소(sparse) 벡터 = 희소 행렬 : 전체 요소 중 단 하나의 값만 1

### 과정
# pip install konlpy
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = '오늘 날씨는 구름이 많아요.'

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

['오늘', '날씨', '구름']


In [27]:
# 단어 사전 구축 
dics = {}
for word in nouns:
  if word not in dics.keys():
    dics[word] = len(dics) # 단어별 인덱스 부여
print(dics)

{'오늘': 0, '날씨': 1, '구름': 2}


In [28]:
# 원-핫 인코딩
nb_classes = len(dics) # 원-핫 벡터 차원의 크기 = 단어 수
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [29]:
# 2. Word2Vec : 분산 표현(distributed representation) = 밀집(dense) 표현

from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

def read_review_data(filename):
  with open(filename, 'r', encoding='UTF8') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:] # 헤더 제외
  return data

# 학습 시간 측정 시작
start = time.time() 

review_data = read_review_data('./data/ratings.txt')
print(len(review_data)) # 리뷰 데이터 전체 개수
print('소요 시간 : ', time.time() - start) # 초

200000
소요 시간 :  0.5178227424621582


In [30]:
komoran = Komoran()

# 명사만 추출
docs = [komoran.nouns(sentence[1]) for sentence in review_data]

# 모델 학습
model = Word2Vec(sentences=docs, # 학습에 필요한 문장 데이터, 모델의 입력값
                 size=200,       # 벡터의 차원, 크기
                 window=4,       # 주변 단어 크기 (-+4)
                 hs=1,           # softmax 사용
                 min_count=2,    # 빈도수 2이하의 단어들은 학습X
                 sg=1)           # skip_gram 모델 <-> 0 : CBOW 모델

# 모델 저장
model.save('nvmc_model')

print('학습된 전체 말뭉치 수 : ', model.corpus_count)
print('전체 단어 수 : ', model.corpus_total_words)

학습된 전체 말뭉치 수 :  200000
전체 단어 수 :  1076896


In [31]:
# model = Word2Vec.load('nvmc.model')
print(model.corpus_total_words)

1076896


In [32]:
# '사랑'의 단어 임베딩 벡터
print('사랑 : ', model.wv['사랑'])

사랑 :  [ 6.14345670e-02  2.52279639e-01  2.99800009e-01  1.80889145e-01
 -1.46754071e-01  3.01284879e-01  2.03659415e-01 -8.12775940e-02
 -1.09657668e-01 -2.12048292e-01 -3.12379926e-01  2.29303777e-01
  9.04875547e-02 -2.63465077e-01 -1.50522575e-01 -4.72635955e-01
 -1.03590719e-01  6.44945651e-02 -8.35087299e-02  2.88091689e-01
  1.93245471e-01  2.67339230e-01  8.45760852e-02  1.45005673e-01
 -1.29181519e-01  5.89859374e-02 -1.10271350e-02  1.64079368e-01
  4.20050591e-01 -1.18626550e-01 -8.64606574e-02 -2.39835218e-01
 -3.33553813e-02  3.76653671e-01 -1.06529303e-01 -3.30360204e-01
 -1.82040930e-01  8.04337393e-03 -1.26872748e-01 -4.95716836e-03
 -1.78446800e-01 -1.65993571e-01 -1.66140884e-01  1.12039581e-01
  3.10913264e-03 -2.15538770e-01  2.19391033e-01  3.67155373e-01
 -3.60003710e-01  2.55180687e-01 -1.54547796e-01 -2.19994232e-01
  1.26030654e-01  2.06073970e-01 -1.08034149e-01  4.54661340e-01
  1.39394850e-01  2.55178750e-01  3.89533420e-03  8.40348750e-02
 -2.43735701e-01  8

In [33]:
# 단어 유사도 계산
print('일요일 = 월요일 :', model.wv.similarity(w1='일요일', w2='월요일'))
print('안성기 = 배우 :', model.wv.similarity(w1='안성기', w2='배우'))
print('대기업 = 삼성 :', model.wv.similarity(w1='대기업', w2='삼성'))
print('일요일 = 삼성 :', model.wv.similarity(w1='일요일', w2='삼성'))
print('히어로 = 삼성 :', model.wv.similarity(w1='히어로', w2='삼성'))

일요일 = 월요일 : 0.6527798
안성기 = 배우 : 0.5747531
대기업 = 삼성 : 0.6002355
일요일 = 삼성 : 0.27493793
히어로 = 삼성 : 0.18756816


In [34]:
# 가장 유사한 단어 추출
print(model.wv.most_similar('안성기', topn=5))
print(model.wv.most_similar('시리즈', topn=5))

[('씨야', 0.7425095438957214), ('장미희', 0.7280349731445312), ('정려원', 0.7247235774993896), ('지진희', 0.7102241516113281), ('김갑수', 0.7100939154624939)]
[('캐리비안의 해적', 0.6526104211807251), ('X맨', 0.6381583213806152), ('러시아워', 0.6327197551727295), ('더 울버린', 0.6301442980766296), ('잭 라이언', 0.6222367286682129)]
